In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
from torch.utils.data import Dataset
import os
import copy
from tensorboardX import SummaryWriter
from PIL import Image
import torchvision.transforms as transforms

plt.ion()   # interactive mode
%matplotlib inline

In [2]:
class CustomDataset(Dataset):
    def __init__(self, root='D:\\Research\\data\\UCMerced_LandUse\\Images', transforms_=None, idx=[]):
        self.transform = transforms_
        self.files = []

        for path, subdirs, files in os.walk(root):
            for name in files:
                self.files.append(os.path.join(path, name))
        
        self.files =  [self.files[i] for i in idx]
        self.n = len(self.files)
        
        classes = os.listdir(root)
        self.classes = {}
        
        for i, cls in enumerate(classes):
            self.classes[cls] = i


    def __getitem__(self, index):
        filepath = self.files[index % len(self.files)]

        cls_name = filepath.split('\\')[-2]
        img = self.transform(Image.open(filepath))
        label = self.classes[cls_name]

        return img, label

    def __len__(self):
        return len(self.files)

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
}

class_names = 21
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset_sizes = {'val': 210, 'train': 2100-210}


In [4]:
from tqdm import tqdm

def train_model(model, criterion, optimizer, scheduler, dataloaders, writer=None, i=0,  num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            

            writer.add_scalar(phase+"/loss", epoch_loss, epoch)
            writer.add_scalar(phase+"/acc", epoch_acc, epoch)


            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                # torch.save(model.state_dict(), 'saved/resnet_'+str(i)+'.pt')

        print()
        

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_acc

# DenseNet

In [5]:
accuracies = []
index = list(np.random.permutation(2100))


for i in range(5):
    idx = {'val': index[210*i:210*(i+1)], 'train': index[:i*210]+index[((i+1)*210) -1:]}

    dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=16,
                                             shuffle=True)
              for x in ['train', 'val']}

    model_ft = models.densenet161(pretrained=True)
    num_ftrs = model_ft.classifier.in_features
    model_ft.classifier = nn.Linear(num_ftrs, class_names)

    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0005, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=6, gamma=0.1)
    writer = SummaryWriter('./logs/densenet161_kfold_'+str(i))

    model_ft, best_acc = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                                     dataloaders, writer, i, num_epochs=20)
    
    accuracies.append(best_acc.item())
    print("\n\n\n------------------------------------\n Accuracies ", accuracies)
    print("\n\n")

Epoch 0/19
----------


100%|████████████████████████████████████████████████████████████████████████████████| 119/119 [01:15<00:00,  1.57it/s]


train Loss: 1.9881 Acc: 0.5704


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:02<00:00,  4.88it/s]


val Loss: 0.7769 Acc: 0.9048

Epoch 1/19
----------


...

val Loss: 0.1418 Acc: 0.9571

Epoch 19/19
----------


100%|████████████████████████████████████████████████████████████████████████████████| 119/119 [01:15<00:00,  1.59it/s]


train Loss: 0.0642 Acc: 0.9942


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:02<00:00,  5.04it/s]


val Loss: 0.1510 Acc: 0.9619

Training complete in 25m 53s
Best val Acc: 0.961905



------------------------------------
 Accuracies  [0.9904761904761905, 0.9952380952380954, 0.9904761904761905, 0.9952380952380954, 0.961904761904762]





# Resnet 152

In [6]:
accuracies = []
index = list(np.random.permutation(2100))


for i in range(5):
    idx = {'val': index[210*i:210*(i+1)], 'train': index[:i*210]+index[((i+1)*210) -1:]}

    dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=16,
                                             shuffle=True)
              for x in ['train', 'val']}

    
    model_ft = models.resnet152(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, class_names)
    
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0005, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=6, gamma=0.1)
    writer = SummaryWriter('./logs/densenet161_kfold_'+str(i))

    model_ft, best_acc = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                                     dataloaders, writer, i, num_epochs=20)
    
    accuracies.append(best_acc.item())
    print("\n\n\n------------------------------------\n Accuracies ", accuracies)
    print("\n\n")

Epoch 0/19
----------


100%|████████████████████████████████████████████████████████████████████████████████| 119/119 [01:16<00:00,  1.56it/s]


train Loss: 2.1332 Acc: 0.4836


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:02<00:00,  4.99it/s]


val Loss: 1.0579 Acc: 0.8667

Epoch 1/19
----------


...

val Loss: 0.0577 Acc: 0.9762

Epoch 19/19
----------


100%|████████████████████████████████████████████████████████████████████████████████| 119/119 [01:15<00:00,  1.57it/s]


train Loss: 0.0639 Acc: 0.9942


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:02<00:00,  5.01it/s]


val Loss: 0.0494 Acc: 0.9905

Training complete in 26m 21s
Best val Acc: 0.990476



------------------------------------
 Accuracies  [0.9952380952380954, 0.961904761904762, 0.980952380952381, 0.9904761904761905, 0.9904761904761905]





# VGG

In [7]:
class classifier(nn.Module):
    def __init__(self):
        super(classifier, self).__init__()
        self.model = models.vgg19_bn(pretrained=True)
        self.fc = nn.Linear(in_features=1000, out_features=class_names,bias=True)
        
    def forward(self, x):
        x = self.model(x)
        return self.fc(x)


In [8]:
accuracies = []
index = list(np.random.permutation(2100))


for i in range(2):
    idx = {'val': index[210*i:210*(i+1)], 'train': index[:i*210]+index[((i+1)*210) -1:]}

    dataloaders = {x: torch.utils.data.DataLoader(CustomDataset(transforms_=data_transforms[x], idx=idx[x]), batch_size=8,
                                             shuffle=True)
              for x in ['train', 'val']}
    
    model_ft = classifier()

    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0005, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=6, gamma=0.1)
    writer = SummaryWriter('./logs/densenet161_kfold_'+str(i))

    model_ft, best_acc = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                                     dataloaders, writer, i, num_epochs=25)
    
    accuracies.append(best_acc.item())
    print("\n\n\n------------------------------------\n Accuracies ", accuracies)
    print("\n\n")

Epoch 0/24
----------


100%|████████████████████████████████████████████████████████████████████████████████| 237/237 [01:47<00:00,  2.21it/s]


train Loss: 0.9733 Acc: 0.6926


100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:02<00:00,  9.26it/s]


val Loss: 0.2763 Acc: 0.9048

Epoch 1/24
----------


...

val Loss: 0.1504 Acc: 0.9714

Epoch 24/24
----------


100%|████████████████████████████████████████████████████████████████████████████████| 237/237 [01:45<00:00,  2.24it/s]


train Loss: 0.0213 Acc: 0.9952


100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:02<00:00,  9.27it/s]


val Loss: 0.1358 Acc: 0.9714

Training complete in 43m 0s
Best val Acc: 0.980952



------------------------------------
 Accuracies  [0.9952380952380954, 0.980952380952381]



